###Ingest constructor.json file

####Step 1 - Read the JSON file using the spark dataframe reader 

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
constructor_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [0]:
constructor_df = spark.read \
.schema(constructor_schema)\
.json(f"{raw_folder_path}/constructors.json")

In [0]:
display(constructor_df)

#### Step 2 - Drop unwanted columns from the dataframe

In [0]:
constructor_dropped_df = constructor_df.drop("url")

#### Step 3 - Rename columns and add ingestion date

In [0]:
constructor_final_df = add_ingestion_date(constructor_dropped_df) \
                                             .withColumnRenamed("constructorRef", "constructor_ref") \
                                             .withColumn("data_source", lit(v_data_source))

In [0]:
display(constructor_final_df)

#### Step 4 - Write into a parquet file

In [0]:
constructor_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.constructors")

In [0]:
display(spark.read.parquet(f"{processed_folder_path}/constructors"))

In [0]:
dbutils.notebook.exit("Success")